<a href="https://colab.research.google.com/github/Kanishka-Reddy/MNIST/blob/main/RNN-Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install ncps

import torch
import torch.nn as nn
import torch.nn.functional as F
import math, copy, time
import numpy as np

from ncps.torch import CfC, LTC
from ncps.wirings import AutoNCP


def clones(module, N):
    "Produce N identical layers."
    return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

class LayerNorm(nn.Module):
    "Construct a layernorm module."
    def __init__(self, features, eps=1e-6):
        super(LayerNorm, self).__init__()
        self.a_2 = nn.Parameter(torch.ones(features))
        self.b_2 = nn.Parameter(torch.zeros(features))
        self.eps = eps

    def forward(self, x):
        mean = x.mean(-1, keepdim=True)
        std = x.std(-1, keepdim=True)
        return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

class SublayerConnection(nn.Module):
    """
    A residual connection followed by a layer norm.
    Note for code simplicity the norm is first as opposed to last.
    """
    def __init__(self, size, dropout):
        super(SublayerConnection, self).__init__()
        self.norm = LayerNorm(size)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, sublayer):
        "Apply residual connection to any sublayer with the same size."
        return x + self.dropout(sublayer(self.norm(x)))


class PositionwiseFeedForward(nn.Module):
    "Implements FFN equation."
    def __init__(self, d_model, dropout=0.1):
        super(PositionwiseFeedForward, self).__init__()
        d_ff = d_model * 4
        self.w_1 = nn.Linear(d_model, d_ff)
        self.w_2 = nn.Linear(d_ff, d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x):
        return self.w_2(self.dropout(F.relu(self.w_1(x))))


def attention(query, key, value, mask=None, dropout=None):
    """
        Compute 'Scaled Dot Product Attention'
        query, key, value : batch_size, n_head, seq_len, dim of space
    """
 
    d_k = query.size(-1)
    # scores: batch_size, n_head, seq_len, seq_len
    scores = torch.matmul(query, key.transpose(-2, -1)) \
             / math.sqrt(d_k)
   
    if mask is not None:
        scores = scores.masked_fill(mask==0, -1e9)


    p_attn = F.softmax(scores, dim = -1)
    if dropout is not None:
        p_attn = dropout(p_attn)
    return torch.matmul(p_attn, value), p_attn


class MHPooling(nn.Module):
    def __init__(self, d_model, h, dropout=0.1):
        "Take in model size and number of heads."
        super(MHPooling, self).__init__()
        assert d_model % h == 0
        # We assume d_v always equals d_k
        self.d_k = d_model // h
        self.h = h
        self.linears = clones(nn.Linear(d_model, d_model), 4)
        self.attn = None
        self.dropout = nn.Dropout(p=dropout)

        #auto-regressive
        attn_shape = (1, 3000, 3000)
        subsequent_mask =  np.triu(np.ones(attn_shape), k=1).astype('uint8')
        self.mask = (torch.from_numpy(subsequent_mask) == 0).unsqueeze(1).cuda()
        
    def forward(self, x):
        "Implements Figure 2"

        nbatches, seq_len, d_model = x.shape
        
        # 1) Do all the linear projections in batch from d_model => h x d_k 
        query, key, value = \
            [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
             for l, x in zip(self.linears, (x, x, x))]
        
        # 2) Apply attention on all the projected vectors in batch. 
        x, self.attn = attention(query, key, value, mask=self.mask[:,:, :seq_len, :seq_len], 
                                 dropout=self.dropout)
        
        # 3) "Concat" using a view and apply a final linear. 
        x = x.transpose(1, 2).contiguous() \
             .view(nbatches, -1, self.h * self.d_k)
        return self.linears[-1](x)

class LocalRNN(nn.Module):
    def __init__(self, input_dim, output_dim, rnn_type, ksize, dropout):
        super(LocalRNN, self).__init__()
        """
        LocalRNN structure
        """
        self.ksize = ksize
        self.wiring = AutoNCP(36, output_dim)  # 36 neurons, output_dim outputs

        # if rnn_type == 'LTC':
        #     self.rnn = LTC(input_size=output_dim, units=self.wiring, batch_first=True)
        # else:
        #     self.rnn = CfC(input_size=output_dim, units=self.wiring, batch_first=True)

        if rnn_type == 'GRU':
            self.rnn = nn.GRU(output_dim, output_dim, batch_first=True)
        elif rnn_type == 'LSTM':
            self.rnn = nn.LSTM(output_dim, output_dim, batch_first=True)
        else:
            self.rnn = nn.RNN(output_dim, output_dim, batch_first=True)

        self.output = nn.Sequential(nn.Linear(output_dim, output_dim), nn.ReLU())

        # To speed up
        idx = [i for j in range(self.ksize-1,10000,1) for i in range(j-(self.ksize-1),j+1,1)]
        self.select_index = torch.LongTensor(idx).cuda()
        self.zeros = torch.zeros((self.ksize-1, input_dim)).cuda()

    def forward(self, x):
        nbatches, l, input_dim = x.shape
        x = self.get_K(x) # b x seq_len x ksize x d_model
        batch, l, ksize, d_model = x.shape
        h = self.rnn(x.view(-1, self.ksize, d_model))[0][:,-1,:]
        return h.view(batch, l, d_model)

    def get_K(self, x):
        batch_size, l, d_model = x.shape
        zeros = self.zeros.unsqueeze(0).repeat(batch_size, 1, 1)
        x = torch.cat((zeros, x), dim=1)
        key = torch.index_select(x, 1, self.select_index[:self.ksize*l])
        key = key.reshape(batch_size, l, self.ksize, -1)
        return key


class LocalRNNLayer(nn.Module):
    "Encoder is made up of attconv and feed forward (defined below)"
    def __init__(self, input_dim, output_dim, rnn_type, ksize, dropout):
        super(LocalRNNLayer, self).__init__()
        self.local_rnn = LocalRNN(input_dim, output_dim, rnn_type, ksize, dropout)
        self.connection = SublayerConnection(output_dim, dropout)

    def forward(self, x):
        "Follow Figure 1 (left) for connections."
        x = self.connection(x, self.local_rnn)
        return x



class Block(nn.Module):
    """
    One Block
    """
    def __init__(self, input_dim, output_dim, rnn_type, ksize, N, h, dropout):
        super(Block, self).__init__()
        self.layers = clones(
            LocalRNNLayer(input_dim, output_dim, rnn_type, ksize, dropout), N)
        self.connections = clones(SublayerConnection(output_dim, dropout), 2)
        self.pooling = MHPooling(input_dim, h, dropout)
        self.feed_forward = PositionwiseFeedForward(input_dim, dropout)

    def forward(self, x):
        n, l, d = x.shape
        for i, layer in enumerate(self.layers):
            x = layer(x)
        x = self.connections[0](x, self.pooling)
        x = self.connections[1](x, self.feed_forward)
        return x
    


class RTransformer(nn.Module):
    """
    The overall model
    """
    def __init__(self, d_model, rnn_type, ksize, n_level, n, h, dropout):
        super(RTransformer, self).__init__()
        N = n
        self.d_model = d_model
        self.dropout = nn.Dropout(dropout)
        self.norm = LayerNorm(d_model)
        self.feed_forward = PositionwiseFeedForward(d_model, dropout)
        
        layers = []
        for i in range(n_level):
            layers.append(
                Block(d_model, d_model, rnn_type, ksize, N=N, h=h, dropout=dropout))
        self.forward_net = nn.Sequential(*layers) 

    def forward(self, x):
        x = self.forward_net(x)
        return x


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import torch.nn.functional as F
import os, sys
from torch import nn
base_path = os.path.dirname(os.path.realpath(sys.executable))
sys.path.append(os.path.join(base_path,'../models'))


class RT(nn.Module):
    def __init__(self, input_size, d_model, output_size, h, rnn_type, ksize, n_level, n, dropout=0.2, emb_dropout=0.2):
        super(RT, self).__init__()
        self.encoder = nn.Linear(input_size, d_model)
        self.rt = RTransformer(d_model, rnn_type, ksize, n_level, n, h, dropout)
        self.linear = nn.Linear(d_model, output_size)

    def forward(self, x):
        """Inputs have to have dimension (N, C_in, L_in)"""
        x = x.transpose(-2,-1)
        x = self.encoder(x)
        x = self.rt(x)  # input should have dimension (N, C, L)
        x = x.transpose(-2,-1)
        o = self.linear(x[:, :, -1])
        return F.log_softmax(o, dim=1)

In [3]:
import torch
from torchvision import datasets, transforms

def data_generator(root, batch_size):
    train_set = datasets.MNIST(root=root, train=True, download=True,
                               transform=transforms.Compose([
                                   transforms.ToTensor(),
                                   transforms.Normalize((0.1307,), (0.3081,))
                               ]))
    test_set = datasets.MNIST(root=root, train=False, download=True,
                              transform=transforms.Compose([
                                  transforms.ToTensor(),
                                  transforms.Normalize((0.1307,), (0.3081,))
                              ]))

    train_loader = torch.utils.data.DataLoader(train_set, batch_size=batch_size)
    test_loader = torch.utils.data.DataLoader(test_set, batch_size=batch_size)
    return train_loader, test_loader

In [4]:
import os 
dir_path = os.path.dirname(os.path.realpath(sys.executable))
print(dir_path)

/usr/bin


In [5]:
import os
  
# Directory
directory = "output"
d2 = "data"
  
# Parent Directory path
parent_dir = "/usr/bin"
  
# Path
path = os.path.join(parent_dir, directory)
path2 = os.path.join(parent_dir, d2)
# Create the directory
# 'GeeksForGeeks' in
# '/home / User / Documents'
if not os.path.exists(path):
    os.makedirs(path)
  
if not os.path.exists(path2):
    os.makedirs(path2)



In [6]:
import torch
import torch.optim as optim
import torch.nn.functional as F
import sys, os
sys.path.append("../../")

import numpy as np
import argparse

parser = argparse.ArgumentParser()
parser.add_argument('--cuda', action='store_false')
parser.add_argument('--batch_size', type=int, default=16)
parser.add_argument('--dropout', type=float, default=0.05)
parser.add_argument('--clip', type=float, default=-1,
                    help='gradient clip, -1 means no clip (default: -1)')
parser.add_argument('--epochs', type=int, default=5)
parser.add_argument('--ksize', type=int, default=7)
parser.add_argument('--n_level', type=int, default=4)
parser.add_argument('--log-interval', type=int, default=100, metavar='N')
parser.add_argument('--lr', type=float, default=0.001)
parser.add_argument('--optim', type=str, default='Adam')
parser.add_argument('--rnn_type', type=str, default='LTC')
parser.add_argument('--d_model', type=int, default=32)
parser.add_argument('--n', type=int, default=2)
parser.add_argument('--h', type=int, default=2)
parser.add_argument('--seed', type=int, default=1111)
parser.add_argument('--permute', action='store_true', default=False)

# args = parser.parse_args()
args, unknown = parser.parse_known_args()
np.random.seed(args.seed)
torch.manual_seed(args.seed)
device = torch.device("cuda")

base_path = os.path.dirname(os.path.realpath(sys.executable))
root = os.path.join(base_path,'data')
s_dir = os.path.join(base_path,'output/')


batch_size = args.batch_size
n_classes = 10
input_channels = 1
seq_length = int(784 / input_channels)
epochs = args.epochs
steps = 0
print(args)

train_loader, test_loader = data_generator(root, batch_size)

model = RT(input_channels, args.d_model, n_classes, h=args.h, rnn_type=args.rnn_type, ksize=args.ksize, 
    n_level=args.n_level, n=args.n, dropout=args.dropout, emb_dropout=args.dropout)

model.to(device)

model_name = "d_{}_h_{}_t_{}_ksize_{}_level_{}_n_{}_lr_{}_dropout_{}".format(
            args.d_model, args.h, args.rnn_type, args.ksize, 
            args.n_level, args.n, args.lr, args.dropout)

newpath = r'C:\Program Files\arbitrary' 
if not os.path.exists(newpath):
    os.makedirs(newpath)
  
message_filename = s_dir + 'r_' + model_name + '.txt'
model_filename = s_dir + 'm_' + model_name + '.pt'
with open(message_filename, 'w') as out:
    out.write('start\n')


lr = args.lr
optimizer = getattr(optim, args.optim)(model.parameters(), lr=lr)

def save(model, save_filename):
    with open(save_filename, "wb") as f:
        torch.save(model, f)
    print('Saved as %s' % save_filename)


def output_s(message, save_filename):
    print (message)
    with open(save_filename, 'a') as out:
        out.write(message + '\n')

def train(ep):
    global steps
    train_loss = 0
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        if args.cuda: data, target = data.cuda(), target.cuda()
        data = data.view(-1, input_channels, seq_length)
        if args.permute:
            data = data[:, :, permute]
        data, target = data, target
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        if args.clip > 0:
            torch.nn.utils.clip_grad_norm_(model.parameters(), args.clip)
        optimizer.step()
        train_loss += loss
        steps += seq_length
        if batch_idx > 0 and batch_idx % args.log_interval == 0:
            message = ('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tSteps: {}'.format(
                ep, batch_idx * batch_size, len(train_loader.dataset),
                100. * batch_idx / len(train_loader), train_loss.item()/args.log_interval, steps))
            output_s(message, message_filename)
            train_loss = 0


def test():
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            if args.cuda:
                data, target = data.cuda(), target.cuda()
            data = data.view(-1, input_channels, seq_length)
            if args.permute:
                data = data[:, :, permute]
            data, target = data,  target
            output = model(data)
            test_loss += F.nll_loss(output, target, size_average=False).item()
            pred = output.data.max(1, keepdim=True)[1]
            correct += pred.eq(target.data.view_as(pred)).cpu().sum()

        test_loss /= len(test_loader.dataset)
        message = ('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset),
            100. * correct / len(test_loader.dataset)))
        output_s(message, message_filename)
        return test_loss


if __name__ == "__main__":
    for epoch in range(1, epochs+1):
        train(epoch)
        test()
        if epoch % 10 == 0:
            lr /= 10
            for param_group in optimizer.param_groups:
                param_group['lr'] = lr

Namespace(batch_size=16, clip=-1, cuda=True, d_model=32, dropout=0.05, epochs=5, h=2, ksize=7, log_interval=100, lr=0.001, n=2, n_level=4, optim='Adam', permute=False, rnn_type='LTC', seed=1111)
Train Epoch: 1 [1600/60000 (3%)]	Loss: 2.423717	Steps: 79184
Train Epoch: 1 [3200/60000 (5%)]	Loss: 2.215272	Steps: 157584
Train Epoch: 1 [4800/60000 (8%)]	Loss: 2.052059	Steps: 235984
Train Epoch: 1 [6400/60000 (11%)]	Loss: 1.755487	Steps: 314384
Train Epoch: 1 [8000/60000 (13%)]	Loss: 1.541598	Steps: 392784
Train Epoch: 1 [9600/60000 (16%)]	Loss: 1.292287	Steps: 471184
Train Epoch: 1 [11200/60000 (19%)]	Loss: 1.087588	Steps: 549584
Train Epoch: 1 [12800/60000 (21%)]	Loss: 1.019357	Steps: 627984
Train Epoch: 1 [14400/60000 (24%)]	Loss: 1.037994	Steps: 706384
Train Epoch: 1 [16000/60000 (27%)]	Loss: 0.879296	Steps: 784784
Train Epoch: 1 [17600/60000 (29%)]	Loss: 0.838662	Steps: 863184
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.729116	Steps: 941584
Train Epoch: 1 [20800/60000 (35%)]	Loss: 0.7442

/usr/local/lib/python3.8/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set: Average loss: 0.4218, Accuracy: 8584/10000 (86%)

Train Epoch: 2 [1600/60000 (3%)]	Loss: 0.462453	Steps: 3019184
Train Epoch: 2 [3200/60000 (5%)]	Loss: 0.334685	Steps: 3097584
Train Epoch: 2 [4800/60000 (8%)]	Loss: 0.393604	Steps: 3175984
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.425700	Steps: 3254384
Train Epoch: 2 [8000/60000 (13%)]	Loss: 0.441730	Steps: 3332784
Train Epoch: 2 [9600/60000 (16%)]	Loss: 0.447608	Steps: 3411184
Train Epoch: 2 [11200/60000 (19%)]	Loss: 0.355446	Steps: 3489584
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.358223	Steps: 3567984
Train Epoch: 2 [14400/60000 (24%)]	Loss: 0.393927	Steps: 3646384
Train Epoch: 2 [16000/60000 (27%)]	Loss: 0.350214	Steps: 3724784
Train Epoch: 2 [17600/60000 (29%)]	Loss: 0.360755	Steps: 3803184
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.323068	Steps: 3881584
Train Epoch: 2 [20800/60000 (35%)]	Loss: 0.298745	Steps: 3959984
Train Epoch: 2 [22400/60000 (37%)]	Loss: 0.261055	Steps: 4038384
Train Epoch: 2 [24000/60000 (40%)]	Los

In [7]:
from google.colab import drive
drive.mount('/content/drive')

model_save_name = 'MNIST_classifier.pt'
path = F"/content/drive/My Drive/{model_save_name}" 
torch.save(model.state_dict(), path)

In [22]:
test_set = datasets.MNIST(root=root, train=False, download=True)
t_set = datasets.MNIST(root=root, train=False, download=True,
                              transform=transforms.Compose([
                                  transforms.ToTensor(),
                                  transforms.Normalize((0.1307,), (0.3081,))
                              ]))
data = t_set[4][0].cuda()
data = data.view(-1, input_channels, seq_length)
output = model(data)
pred = output.data.max(1, keepdim=True)[1]

display(test_set[4][0])
print(pred)

tensor([[4]], device='cuda:0')
